In [3]:
# 라이브러리 불러오기
import os
import numpy as np
import pandas as pd
%tensorflow_version 1.x  # 텐서플로우 1.x 버전 사용
import tensorflow as tf
tf.__version__

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x  # 텐서플로우 1.x 버전 사용`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


'1.15.2'

In [2]:
# 데이터 가져오기 (안경잡이 개발자 블로그)
!wget https://ndb796.tistory.com/attachment/cfile3.uf@994519435B7D9B9F150FF5.csv

--2020-04-16 04:58:55--  https://ndb796.tistory.com/attachment/cfile3.uf@994519435B7D9B9F150FF5.csv
Resolving ndb796.tistory.com (ndb796.tistory.com)... 211.231.99.250
Connecting to ndb796.tistory.com (ndb796.tistory.com)|211.231.99.250|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://t1.daumcdn.net/cfile/tistory/994519435B7D9B9F15?download [following]
--2020-04-16 04:58:55--  https://t1.daumcdn.net/cfile/tistory/994519435B7D9B9F15?download
Resolving t1.daumcdn.net (t1.daumcdn.net)... 210.61.207.152
Connecting to t1.daumcdn.net (t1.daumcdn.net)|210.61.207.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93046 (91K) [text/csv]
Saving to: ‘cfile3.uf@994519435B7D9B9F150FF5.csv’

cfile3.uf@994519435 100%[===================>]  90.87K   516KB/s    in 0.2s    

2020-04-16 04:58:56 (516 KB/s) - ‘cfile3.uf@994519435B7D9B9F150FF5.csv’ saved [93046/93046]



In [0]:
# 파일명 바꾸기
os.rename('cfile3.uf@994519435B7D9B9F150FF5.csv', 'raw_data.csv')

In [0]:
# 데이터프레임 불러오기
raw_df = pd.read_csv('raw_data.csv', sep=',')

In [8]:
# 데이터 확인
raw_df.head()

,year,avgTemp,minTemp,maxTemp,rainFall,avgPrice
0,20100101,-4.9,-11.0,0.9,0.0,2123
1,20100102,-3.1,-5.5,5.5,0.8,2123
2,20100103,-2.9,-6.9,1.4,0.0,2123
3,20100104,-1.8,-5.1,2.2,5.9,2020
4,20100105,-5.2,-8.7,-1.8,0.7,2060


In [9]:
 # 데이터 크기 확인
 raw_df.shape

(2922, 6)

In [10]:
# 모든 변수 확인
raw_df.columns

Index(['year', 'avgTemp', 'minTemp', 'maxTemp', 'rainFall', 'avgPrice'], dtype='object')

### 데이터 설명

- 출처 1 : 일별 기상 정보 > [기상자료개방포털 다중지점통계](https://data.kma.go.kr/climate/StatisticsDivision/selectStatisticsDivision.do?pgmNo=158)
  - year : 일시
  - avgTemp : 평균기온
  - minTemp : 최저기온 평균
  - maxTemp : 최저기온
  - rainFall : 강수량
- 출처 2 : 배추 가격 정보 > [농산물 유통 정보](https://www.kamis.or.kr/customer/price/retail/period.do?action=daily)
  - avgPrice : 평균가격

In [0]:
# 데이터프레임 -> 배열
raw_array = np.array(raw_df, dtype=np.float32)

In [12]:
# 독립변수 : 기상요인 ['avgTemp', 'minTemp', 'maxTemp', 'rainFall'] 
x_data = raw_array[:, 1:-1]
x_data.shape

(2922, 4)

In [0]:
# 종속변수 : 배추가격 ['avgPrice']
y_data = raw_array[:, [-1]]

In [0]:
# 초기화
model = tf.global_variables_initializer();

# 플레이스 홀더 설정 : 독립변수, 종속변수, 가중치, 편향
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.random_normal([4, 1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")

In [0]:
# 가설 설정
hypothesis = tf.matmul(X, W) + b

# 비용 함수 설정
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# 최적화 함수 설정
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.000005)
train = optimizer.minimize(cost)

In [20]:
# 세션 생성
with tf.Session() as sess:
  # 글로벌 변수를 초기화
  sess.run(tf.global_variables_initializer())

  # 학습 수행
  for step in range(30001):
    cost_, hypo_, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    if step % 500 == 0:
      print("#step : ", step, "\t# loss :  ", cost_, "\t# 배추 가격: ", hypo_[0])

  # 학습된 모델 저장
  saver = tf.train.Saver()
  save_path = saver.save(sess, "./saved.cpkt")

#step :  0 	# loss :   12854550.0 	# 배추 가격:  [20.056648]
#step :  500 	# loss :   4184173.8 	# 배추 가격:  [-268.99045]
#step :  1000 	# loss :   3779431.2 	# 배추 가격:  [40.014668]
#step :  1500 	# loss :   3463434.0 	# 배추 가격:  [321.66373]
#step :  2000 	# loss :   3214940.5 	# 배추 가격:  [571.89374]
#step :  2500 	# loss :   3019047.8 	# 배추 가격:  [794.2025]
#step :  3000 	# loss :   2864369.8 	# 배추 가격:  [991.7623]
#step :  3500 	# loss :   2742104.0 	# 배추 가격:  [1167.3636]
#step :  4000 	# loss :   2645384.2 	# 배추 가격:  [1323.4767]
#step :  4500 	# loss :   2568830.8 	# 배추 가격:  [1462.284]
#step :  5000 	# loss :   2508210.0 	# 배추 가격:  [1585.7189]
#step :  5500 	# loss :   2460187.2 	# 배추 가격:  [1695.4932]
#step :  6000 	# loss :   2422127.0 	# 배추 가격:  [1793.1262]
#step :  6500 	# loss :   2391948.5 	# 배추 가격:  [1879.9664]
#step :  7000 	# loss :   2368006.5 	# 배추 가격:  [1957.2107]
#step :  7500 	# loss :   2348998.2 	# 배추 가격:  [2025.9219]
#step :  8000 	# loss :   2333896.2 	# 배추 가격:  [2087.0447]
#s

In [25]:
##### 예측하기 
# 4가지 변수를 입력 받습니다.
print("배추값 예측입니다. 기온 정보를 입력해주세요.")
avg_temp = float(input('평균 온도: '))
min_temp = float(input('최저 온도: '))
max_temp = float(input('최고 온도: '))
rain_fall = float(input('강수량: '))

with tf.Session() as sess:
    # 저장된 학습 모델을 파일로부터 불러옵니다.
    save_path = "./saved.cpkt"
    saver.restore(sess, save_path)

    # 사용자의 입력 값을 이용해 배열을 만듭니다.
    data = ((avg_temp, min_temp, max_temp, rain_fall), )
    arr = np.array(data, dtype=np.float32)

    # 예측을 수행한 뒤에 그 결과를 출력합니다.
    x_data = arr[0:4]
    dict = sess.run(hypothesis, feed_dict={X: x_data})

    print(f'배추 예측값은 {int(dict[0][0])}원 입니다.')

배추값 예측입니다. 기온 정보를 입력해주세요.
평균 온도: 10
최저 온도: 0
최고 온도: 20
강수량: 0
INFO:tensorflow:Restoring parameters from ./saved.cpkt
배추 예측값은 5092원 입니다.
